In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d as g1d
from scipy.stats import norm
import matplotlib.pylab as plt
from scipy.interpolate import interp1d
from matplotlib import cm
cmap_sequence = [cm.winter, cm.autumn, cm.spring, cm.summer]


# Quantifying ISGRI Charge Loss

## Purpose of this section

energy reconstruction below 60 keV is tricky but also most relevant

## Origin of the Low-Energy troubles

ISGRI is able to reconstruct photon interactions with incident energies from from `15 keV` to `800 keV` (in ground calibration and early mission, i.e. before any degradation).
But the degradation is very substantial (in simple terms, about factor 2 in gain in 15 years), and it becomes necessary to establish in-flight calibration already in the first years of observations.

Above `58 keV`, ISGRI background spectra reveal plenty of spectral features which can be interpreted and used to establish energy reconstruction.
In most orbits, `59 keV` complex and `511 keV` line are most pronounced features. They are usually quite sufficient to establish gain with high precision (higher than any systematic uncertainties). Any other lines are useful to verify the model shape.

Below, background spectrum with particularily pronounced background lines, following activation by solar activity.
<table><tr>
<td><img src="https://volodymyrss.github.io/imgb/rev_0132.png.1416053377.0d9b1fa1b3" width="500px"></td>
</tr></table>



## Finding reference for ISGRI low-energy reconstruction

OSA11 charge loss model is fitted to lines at `~60 keV` and `511 keV`.
Trivial extrapolation of the evolution to 20 keV would lead to clear mismatch with ground calibration lines (applicable early mission) and cross-calibration obsevations (applicable late mission).


At ~22 keV there is a Cd line complex ([see](https://xdb.lbl.gov/Section1/Table_1-2.pdf)), but it's hard to use it due to several effects (in the order from more important to less important):
* low energy cut-off by pixel threshold. This effect appears as shift to high energy. The LE threshold can be only reasonably determined with astrophysical source observations.
* energy resolution. Since since the broadened line is cutoff at low energy, it unequally broadens to high energy. Resolution at LE can only be rather approximately determined with astrophysical source observations.
* underlying background is very variable at low energy, and sensitive to CR-induced activation. Also astrophysical sources brighter than 300 mCrab are comparable to background level, and bias energy estimation depending on the source spectrum, usually to high energy. It is possible to exclude particularly anomalous background conditions, but long-term trends are hard to disentangle from other effects. See [there](https://volodymyrss.github.io/imgb/history.html)

For realistic magnitude of these effects, the shifts may lead to apparent line centroid from 20 to 30 keV, for actual line at 22 keV. See example below.

Below, shown separately background and source rates

<table><tr>
<td><img src="https://volodymyrss.github.io/imgb/detelc_decomposed.png.1406121161.88b8ca491b" width="500px"></td>
<td><img src="https://volodymyrss.github.io/imgb/history.png.1406120164.1771a51d05" width="500px"></td>
</tr></table>

<!-- <img src="images/isgri-pixel.svg"> -->

In [ ]:
from IPython.core.display import SVG
SVG(filename='images/isgri-pixel.svg')

In [ ]:
plt.figure(figsize=(10, 7))

en = np.logspace(1.1, 3, 300)


raw_line = norm(22, 0.1).pdf(en)

def leeffi(en, position=15, sharpness=14):
    return (en/position)**sharpness/(1+(en/position)**sharpness)

def observe(raw, sigma, lecutoff):
    return g1d(raw, sigma)*leeffi(en, lecutoff)


for sigma, lecutoff, extrabkg in [
    (10, 20, 0),
    (10, 30, 0),
    (15, 30, 0),    
    (10, 20, 0.05*(en/30)**-2),
    (10, 20, 0.05*(en/30)**-3),
]:
    m = en < 50
    obs_nole = observe(raw_line, sigma, 0.1)
    obs = observe(raw_line + extrabkg, sigma, lecutoff)

    mean = np.sum((en*obs)[m])/np.sum(obs[m])
    width = np.nansum((obs_nole*en**2)[m])/np.nansum(obs_nole[m]) - (np.nansum((obs_nole*en)[m])/np.nansum(obs_nole[m]))**2

    plt.plot(
        en,
        obs,
        label=f"sigma: {sigma} ({width:.1f} keV) lecutoff: {lecutoff} keV: resulting mean {mean:.1f} keV"
    )    



plt.loglog()
plt.legend()

plt.ylim([1e-5, 10])

In [ ]:
import importlib
import sys
import os
import numpy as np
import importlib

os.environ['EDDOSA_TOOLS_DIR'] = os.getcwd() + '/../eddosa_tools/'

sys.path.insert(0, '../dda-ddosa')
sys.path.insert(0, '../dda-eddosa')
sys.path.insert(0, '../eddosa_tools/fit1d')
sys.path.insert(0, '../lut2model/python')

import lut2model
import bipar_model
import eddosa

lut2model.set_verbosity(0)
importlib.reload(bipar_model)


importlib.reload(bipar_model)
det_base = bipar_model.detector.from_lut2_fits('/mnt/sshfs/cdcihn/unsaved/astro/savchenk/data/reduced/ddcache/byrev/0239/FinalizeLUT2.xvbase5.2/4711debf/lut2_1d_final.fits.gz')

print(det_base)

ilut2_base, i1d_base = det_base.generate_approximate_reconstruction(np.logspace(0, 3, 15))

from IPython.core.display import SVG
SVG(filename='images/isgri-pixel.svg')

In [ ]:
#lut2model.render_bipar_m0()

det = det_base.copy()

pha_2d, rt_2d = det.mesh()


bip = np.zeros_like(pha_2d, dtype=float)
    
for en in [22, 59, 150, 511]:
    _bip = bipar_model.make_bipar_monoenergetic(det, en).transpose()
    
    bip += _bip / np.nanquantile(_bip[_bip>_bip.max()/100].flatten(), 0.5)

f, (ax1,ax2) = plt.subplots(2,1, figsize=(15, 8), sharex=True)
ax1.contourf(pha_2d, rt_2d, bip**0.5, levels=np.logspace(-1, 0)*np.nanmax(bip**0.5))

ax2.plot(
    pha_2d[0,:],
    bip[15:30,:].sum(0)
)

plt.semilogx()
ax1.set_xlim([15, 2000])
ax1.set_ylim([16, 200])
plt.subplots_adjust(hspace=0)

ax1.set_title('different lines for baseline model', size=20)
ax1.set_xlabel('PHA (ISGRI Pulse Height channel)', size=14)
ax1.set_ylabel('RT (ISGRI Rise Time channel)', size=14)
ax2.set_ylabel('Stacked in RT 15 - 30', size=14)



In [ ]:
plt.figure(figsize=(12,6))

spec = bip[15:30,:].sum(0)

en_base = i1d_base(pha_2d[0,:])

plt.plot(
    en_base,
    spec,
)

for en in [22, 59, 511, 150]:
    plt.axvline(en, c='r')

plt.semilogx()

plt.xlabel("approximately reconstructed ISGRI energy", size=14)

#TODO: total event number lost to high RT threshold
#TODO: discontinuities from abs edges

In [ ]:
bip = np.zeros_like(pha_2d, dtype=float)

lrt_lines = []

plt.figure(figsize=(15,8))

N=20

energies = [22, 59, 511]

for i_loss, tau_loss_factor in enumerate(np.logspace(-0.5, 0, N)):
    det = det_base.copy()
    det.tau_e = det_base.tau_e * tau_loss_factor

    this_loss_bip = np.zeros_like(bip)

    for en in energies:
        _bip = bipar_model.make_bipar_monoenergetic(det, en).transpose()

        for rt1, rt2 in [(15,30), (50,70)]:
            ltr_bip = _bip[rt1:rt2, :].sum(0)
            lrt_lines.append(
                dict(
                    rt1=rt1,
                    rt2=rt2,
                    energy=en,
                    det=det,
                    mean_pha=np.sum(pha_2d[0,:]*ltr_bip)/np.sum(ltr_bip),
                    _bip=_bip,
                )
                )
        
        this_loss_bip += _bip / np.nanquantile(_bip[_bip>_bip.max()/100].flatten(), 0.5)        
    

    if i_loss % (N-1) == 0:
        bip += this_loss_bip
        spec = this_loss_bip[15:30,:].sum(0)
        m = spec > np.nanmax(spec) * 1e-4

        c=plt.plot(
            pha_2d[0,m],
            spec[m],
            alpha=0.1
        )[0].get_color()

        plt.plot(
            en_base[m],
            spec[m],
            ls="--",
            c=c
        )

        if i_loss == 0:
            for en in energies:
                plt.axvline(en, lw=5, c='r', alpha=0.2)
        
        

plt.title(f'tau_loss and reconstruction with {det_base}')

plt.semilogx()

plt.figure(figsize=(15, 8))
plt.contourf(pha_2d, rt_2d, bip**0.5, levels=np.logspace(-1, 0)*np.nanmax(bip**0.5))

plt.semilogx()
plt.xlim([5, 2000])

plt.xlabel('PHA (ISGRI Pulse Height channel)')
plt.ylabel('RT (ISGRI Rise Time channel)')

# for vl in vlines:
#     plt.axvline(vl)

Charge loss model, in principle, results in a prescription to how low-energy part evolves. This prescription is somewaht different from simple linear change of offset and gain. But not as different as what is observed at low energy.

Prescriptions for evolution at different energies are are shown below.

How is it possible that 60 keV is shifted differently than 30 keV?

* additional unaccounted shift at 511 keV by about XX
* additional unaccounted shift at 59 keV by about XX



In [ ]:
plt.figure(figsize=(15, 10))
offset = -8

for en in set([l['energy'] for l in lrt_lines]):
    c = None
    ls_list=['-','--']
    for rt1 in [15, 50]:
        mean_phas = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == en and l['rt1'] == rt1])
        normal_phas = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == 59 and l['rt1'] == rt1])
        ls=ls_list.pop(0)
        c=plt.plot(
            [-np.log10(l['det'].tau_e) for l in lrt_lines if l['energy'] == en  and l['rt1'] == rt1],
            (mean_phas - det.offset)/(mean_phas - det.offset).max()/(normal_phas - det.offset)*(normal_phas - det.offset).max(),
            label=f"{en} keV rt {rt1} relative to 59 keV",
            c=c,
            ls=ls,
            lw=2
        )[0].get_color()

        c=plt.plot(
            [-np.log10(l['det'].tau_e) for l in lrt_lines if l['energy'] == en  and l['rt1'] == rt1],
            (mean_phas - det.offset)/(mean_phas - det.offset).max(),
            label=f"{en} keV rt {rt1}",
            c=c,
            ls=ls,
            lw=2,
            alpha=0.2            
        )[0].get_color()


plt.legend()
#plt.semilogx()
plt.title("loss of PHA as $\tau_e$ degrades with mission lifetime: ???")
plt.xlabel(r"- log$_{10}~\tau_e$ (grows with mission lifetime)", size=24)
plt.ylabel(r"fraction of PH lost", size=24)

_=plt.yticks(fontsize=18)
_=plt.xticks(fontsize=18)

#TODO: estimate effective offset, gain, gain2

In [ ]:
pha_22_lrt  = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == 22 and l['rt1'] == 15])
pha_59_lrt  = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == 59 and l['rt1'] == 15])
pha_511_lrt  = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == 511 and l['rt1'] == 15])
l_tau_e = np.array([-np.log10(l['det'].tau_e) for l in lrt_lines if l['energy'] == 22  and l['rt1'] == 15])

plt.figure()

# plt.plot(
#     l_tau_e,
#     (pha_59_lrt - pha_22_lrt)/(59-22),
#     label="22 - 59 gain"
# )

# plt.plot(
#     l_tau_e,
#     (pha_511_lrt - pha_59_lrt)/(511-59),
#     label="59 - 511 gain"
# )

# plt.plot(
#     l_tau_e,
#     (pha_59_lrt - pha_22_lrt)/(59-22)/((pha_511_lrt - pha_59_lrt)/(511-59)),
#     label="59 - 511 / 22 - 9 gain"
# )

# plt.axhline(1.05)

he_gain = (pha_511_lrt - pha_59_lrt)/(511-59)

plt.plot(
    l_tau_e,
    pha_59_lrt - he_gain*(59-22),
    label="he-predicted linear"
)


plt.plot(
    l_tau_e,
    pha_22_lrt,
    label="model-preducted"
)


plt.grid()

plt.legend()


# why drop? describe better non-linearity of the LE prediction

# Detector polarization

As discussed above, assuming no polarization, or uniform polarization does not reproduce low-energy response compatible with observations.

Since detector is always polarized, it's hard to distinguish effect of degradation of charge carrior properties due to irradiation from
polarization. 

Especially if the polarization is inhomogenous. And since polarization effect itself is known to depend on irradiation dose itself, just as charge carried properties do.


In [ ]:
bip = np.zeros_like(pha_2d, dtype=float)

lrt_lines = []


f, (ax1,ax2) = plt.subplots(2,1, figsize=(15, 8), sharex=True)

#for tau_loss_factor in np.logspace(-1., 0, 3):
N = 20
for i_loss, V_loss_factor in enumerate(np.logspace(-0.2, 0, N)):
    det = bipar_model.detector()
    #det.tau_e = det_base.tau_e * tau_loss_factor
    det.V = det_base.V * V_loss_factor

    c = None
    label = f"V = {det.V:.2f}"

    for en in [25, 59, 511]:
        _bip = bipar_model.make_bipar_monoenergetic(det, en).transpose()

        for rt1, rt2 in [(15,30), (50,70)]:
            ltr_bip = _bip[rt1:rt2, :].sum(0)
            lrt_lines.append(
                dict(
                    rt1=rt1,
                    rt2=rt2,
                    energy=en,
                    tau_e=det.tau_e,
                    V=det.V,
                    mean_pha=np.sum(pha_2d[0,:]*ltr_bip)/np.sum(ltr_bip)
                )
                )
        
        bip += _bip / np.nanquantile(_bip[_bip>_bip.max()/100].flatten(), 0.5)

        if i_loss % (N-1) == 0:
            ax1.contour(pha_2d, rt_2d, _bip**0.5, levels=np.logspace(-0.5, -0.3, 3)*np.nanmax(_bip**0.5),
                        cmap=cmap_sequence[int(i_loss/(N-1))]
            )
         
            c = ax2.plot(
                pha_2d[0,:],
                _bip[15:30,:].sum(0)/np.nanmax(_bip[15:30,:].sum(0)),
                c=c,
                label=label
            )[0].get_color()
            label = None



#plt.figure(figsize=(15, 8))
#plt.contourf(pha_2d, rt_2d, bip**0.5, levels=np.logspace(-1, 0)*np.nanmax(bip**0.5))

plt.title("field (V, polarization) effect on charge collection")

plt.semilogx()
plt.xlim([30, 2000])
ax1.set_ylim([16, 116])

ax2.legend()

plt.xlabel('PHA (ISGRI Pulse Height channel)')
plt.ylabel('RT (ISGRI Rise Time channel)')

# for vl in lrt_lines:
#     plt.axvline(vl['mean_pha'])

In [ ]:
plt.figure(figsize=(15, 10))

for en in set([l['energy'] for l in lrt_lines]):
    c = None
    ls_list=['-','--']
    for rt1 in [15, 50]:
        normal_phas = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == 59 and l['rt1'] == rt1])
        normal_phas /= np.nanmax(normal_phas)

        mean_phas = np.array([l['mean_pha'] for l in lrt_lines if l['energy'] == en and l['rt1'] == rt1])
        c=plt.plot(
            [l['V'] for l in lrt_lines if l['energy'] == en  and l['rt1'] == rt1],
            mean_phas/np.nanmax(mean_phas)/normal_phas,
            label=f"{en} keV rt {rt1}",
            c=c,
            ls=ls_list.pop(0),
            lw=2
        )[0].get_color()

plt.legend()
#plt.semilogx()
plt.xlabel(r"V", size=24)
plt.ylabel(r"fraction of PH lost normalized to loss at 60 keV", size=24)

ax=plt.gca()
ax.set_xlim(ax.get_xlim()[::-1])

As shown in the picture above, drop of the bias `V` (or, more directly, field `E`) leading to drop in the field in the detector, would cause more rapid loss of gain at low energy than at high energy. Which is the opposite of what is observed. 

High-RT counts correspond to low-depth interactions spread with RT resolution (at low energy) or  actually high-depth interactions. The latter are less affected.

So, some other effect is at play, possibly inhomogenous polarization. It is not immediately clear what kind.

Fortunately, polarization can be also observed directly following detector switch-on. It can not be used to give direct prescription for long-term polarization change, but it can give an insight.


# Mismatch between no-polarization model and in-flight source observations

Observations with cyclotron line sources indicate excessive compression of reconstructed energy scale by up to 10% at 30 keV (see [CC-summary](https://gitlab.astro.unige.ch/integral/cc-workflows/cc-global-summary/-/blob/master/verify.ipynb))

Roughly speaking, the model predicts that 30 keV converts PHA=30 to 2021. But we observe that 30 keV converts to PHA=50. #TODO: think again!

It means that charge collection for 30 keV depositions (which, inevitably, all happen near the detector surface, just as 60 keV depositions) remain more efficient than expected, despite substantial loss of electron lifetime, as seen in charge collection for 59 and 511 keV.

#TODO: insert pic from offsets


When fitting with the model with variable offset by no additional free parameters (no polarization change, for example), this appears as drop in offset.
In principle, offset is expected to be constant.

#TODO: think! 
So the effect is the opposite of what is observed.

* Polarization has the same direction as usual degradation, does not help
* inhomogenous polarization will still affect lRT PHA for LE just as HE lines.

Astro observations seems to suggest that constant offset would work.

#TODO: end-goal here is to show how to:
* enforce constant offset in efficiency, events, response
* centralize response

#  Effects of offset

ISGRI has negative offset. This is complicating reconstruction since it is difficult to characterize behavior near

Since, as explained above, it is not easy to characterize EN-PHA conversion at low energy,

the calibration line evolution as observed indicates

In [ ]:
# TODO: measure or mimick gain from HE line alone, and from LE line alone. find which one mismatches


bip = np.zeros_like(pha_2d, dtype=float)

lrt_lines = []

f, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15, 12), sharex=True)

extra_ticks = []

en_rec = None


#for tau_loss_factor in np.logspace(-1., 0, 3):
N = 5
for i_loss, (comment, V_loss_factor, offset_shift, tau_loss_factor) in enumerate([
            ('loss of tau and offset', 1, -5, 0.7),
            ('loss of tau to match HE', 1, 0, 0.655),
            ('loss of tau to match LE', 1, 0, 0.5),
            #('loss of V', 0.8, 0, 1),
            #('loss of V and tau', 0.9, 0, 0.9),
            #('offset drop', 1, -4, 1),
        ]):
    det = bipar_model.detector()
    #det.tau_e = det_base.tau_e * tau_loss_factor
    det.V = det_base.V * V_loss_factor
    det.offset = det_base.offset + offset_shift
    det.tau_e = det_base.tau_e * tau_loss_factor

    c = None
    label = f"{comment}: V = {det.V:.3g} offset = {det.offset:.3g} $\\tau_e$ = {det.tau_e:.2g}"
    cmap = cmap_sequence.pop(0)
    cmap_sequence.append(cmap)

    if en_rec is None:       
        det_rec = det.copy() 
        ilut2_rec, i1d_rec = det_rec.generate_approximate_reconstruction(np.logspace(0, 3, 15), plot=False)
        en_rec = i1d_rec(pha_2d[0,:])

    for en in [25, 59, 511]:
        _bip = bipar_model.make_bipar_monoenergetic(det, en).transpose()

        for rt1, rt2 in [(15,30), (50,70)]:
            ltr_bip = _bip[rt1:rt2, :].sum(0)
            lrt_lines.append(
                dict(
                    rt1=rt1,
                    rt2=rt2,
                    energy=en,
                    tau_e=det.tau_e,
                    V=det.V,
                    mean_pha=np.sum(pha_2d[0,:]*ltr_bip)/np.sum(ltr_bip)
                )
                )
        
        bip += _bip / np.nanquantile(_bip[_bip>_bip.max()/100].flatten(), 0.5)

        if i_loss % (N-1) == 0 or True:
            ax1.contour(pha_2d, rt_2d, _bip**0.5, levels=np.logspace(-0.5, -0.3, 3)*np.nanmax(_bip**0.5),
                        cmap=cmap
            )
         
            sp = _bip[15:30,:].sum(0)/np.nanmax(_bip[15:30,:].sum(0))
            m = sp > np.nanmax(sp)/1e3
            c = ax2.plot(
                pha_2d[0,:][m],
                sp[m],
                c=c,
                alpha=0.1
            )[0].get_color()
         
            ax2.plot(
                en_rec[m],
                sp[m],
                ls="--",
                label=label,
                c=c
            )
            label = None

            rec_en = np.sum(en_rec[m]*sp[m])/np.sum(sp[m])
            print(f"{comment:30s} rec {rec_en:5.3g} (exp {en:5.3g}) : {((rec_en/en)-1)*100:5.3g} % with", det_rec)
         
            extra_ticks.append(
                np.sum(pha_2d[0,:][m]*sp[m])/np.sum(sp[m])
            )




#plt.figure(figsize=(15, 8))
#plt.contourf(pha_2d, rt_2d, bip**0.5, levels=np.logspace(-1, 0)*np.nanmax(bip**0.5))

plt.semilogx()

#ax2.set_xticks(list(ax2.get_xticks()) + extra_ticks)
ax2.set_xticks(extra_ticks)
#ax2.set_xticklabels(list(ax2.get_xticklabels()) + [str(t) for t in extra_ticks])
ax2.set_xticklabels([f"{t:.4g}" for t in ax2.get_xticks()], rotation='vertical')

ax3.contourf(pha_2d, rt_2d, bip**0.5, levels=np.logspace(-1, 0)*np.nanmax(bip**0.5))


plt.xlim([10, 2000])
ax1.set_ylim([16, 150])

ax2.legend()

plt.subplots_adjust(hspace=0)

plt.xlabel('PHA (ISGRI Pulse Height channel)')
plt.ylabel('RT (ISGRI Rise Time channel)')


# TODO: apply here the D-law

# for vl in lrt_lines:
#     plt.axvline(vl['mean_pha'])

In [ ]:

# TODO:
# * reconstruct with det and morph




It was suggested [[1]](https://hal.archives-ouvertes.fr/hal-00000723/document) that for shallow interractions charge loss is negligible. This is certainly not the case as detector evolves. But it is also likely not the case already for the first in-flight measurements (and possibly for ground calibration).

RT for electrons is smaller than that for holes, but really shallow interractions. 
The lowest RT is achieved for some characteristic depth, dependent on energy.

very shallow interactions have very small RT from holes, 

But, in principle, **low-RT counts always correspond to the same interaction depth, and experience the same field, same ballistic losses. They only differ in the amount of charge.**. It means that they are affected by the same field.

#TODO find point as function of energy. find depth as function of RT range for different energies

In [ ]:
#TODO: deduce gain evolution just from the first line, assuming constant offset.
#TODO: reconstruction with mismatching detector configurations